In [1]:
import os
from random import random

import cv2
import numpy as np
import pandas as pd

import image_processor
import model_builder

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.model_selection import train_test_split

from datetime import datetime

In [2]:
image_size = (32, 32)
input_size = (*image_size, 1)

In [3]:
df_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
            'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
            'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
            'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
            'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [4]:
df_train = image_processor.process_image_data("data/Train", image_size, targets=df_names)

In [5]:
X = df_train.drop(columns=['target'])
y = df_train['target']

In [6]:
X_data = X.values.reshape(-1, *image_size, 1).astype('float32')
y_data = y.factorize()[0]

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, stratify=y_data, random_state=42)

In [14]:
model = model_builder.build_conv_pool_model(input_size, len(df_names))

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=256,
                    callbacks=[early_stopping, tensorboard_callback],
                    validation_data=(X_val, y_val),
                    verbose=1)

Epoch 1/50
187/187 [==============================] - 26s 135ms/step - loss: 1.4036 - accuracy: 0.6247 - val_loss: 4.5518 - val_accuracy: 0.0359
Epoch 2/50
187/187 [==============================] - 26s 140ms/step - loss: 0.5673 - accuracy: 0.8120 - val_loss: 1.2338 - val_accuracy: 0.6499
Epoch 3/50
187/187 [==============================] - 29s 154ms/step - loss: 0.4472 - accuracy: 0.8411 - val_loss: 0.3985 - val_accuracy: 0.8535
Epoch 4/50
187/187 [==============================] - 25s 131ms/step - loss: 0.3720 - accuracy: 0.8602 - val_loss: 0.3083 - val_accuracy: 0.8815
Epoch 5/50
187/187 [==============================] - 26s 140ms/step - loss: 0.3318 - accuracy: 0.8735 - val_loss: 0.2850 - val_accuracy: 0.8904
Epoch 6/50
187/187 [==============================] - 24s 126ms/step - loss: 0.2967 - accuracy: 0.8840 - val_loss: 0.3027 - val_accuracy: 0.8798
Epoch 7/50
187/187 [==============================] - 23s 121ms/step - loss: 0.2735 - accuracy: 0.8900 - val_loss: 0.2459 - val_ac

In [15]:
for epoch, (val_loss, val_acc) in enumerate(zip(history.history['val_loss'], history.history['val_accuracy']), start=1):
    print(f"Epoch {epoch}: val_loss={val_loss:.4f}, val_accuracy={val_acc:.4f}")

score = model.evaluate(X_val, y_val, verbose=0)
print(f"Final Validation Accuracy: {score[1] * 100:.2f}%")
# 91.84

Epoch 1: val_loss=4.5518, val_accuracy=0.0359
Epoch 2: val_loss=1.2338, val_accuracy=0.6499
Epoch 3: val_loss=0.3985, val_accuracy=0.8535
Epoch 4: val_loss=0.3083, val_accuracy=0.8815
Epoch 5: val_loss=0.2850, val_accuracy=0.8904
Epoch 6: val_loss=0.3027, val_accuracy=0.8798
Epoch 7: val_loss=0.2459, val_accuracy=0.9015
Epoch 8: val_loss=0.2603, val_accuracy=0.8949
Epoch 9: val_loss=0.2495, val_accuracy=0.8998
Epoch 10: val_loss=0.2305, val_accuracy=0.9047
Epoch 11: val_loss=0.2317, val_accuracy=0.9103
Epoch 12: val_loss=0.2338, val_accuracy=0.9118
Epoch 13: val_loss=0.2576, val_accuracy=0.8983
Epoch 14: val_loss=0.2232, val_accuracy=0.9156
Epoch 15: val_loss=0.2277, val_accuracy=0.9103
Epoch 16: val_loss=0.2402, val_accuracy=0.9120
Epoch 17: val_loss=0.2092, val_accuracy=0.9205
Epoch 18: val_loss=0.2105, val_accuracy=0.9173
Epoch 19: val_loss=0.2511, val_accuracy=0.9064
Epoch 20: val_loss=0.2310, val_accuracy=0.9109
Epoch 21: val_loss=0.2455, val_accuracy=0.9103
Epoch 22: val_loss=0.2

In [10]:
test_df = df_test = image_processor.process_image_data("data/TestData", image_size)

In [11]:
X_test = test_df.iloc[:, 1:]
X_test_prepared = X_test.values.reshape(-1, *image_size, 1).astype('float32')

pred = model.predict(X_test_prepared)
predicted_classes = pred.argmax(axis=1)
predicted_labels = [df_names[i] for i in predicted_classes]

test_df['pred'] = predicted_labels

222/222 [==============================] - 2s 7ms/step


In [16]:
model.save("models/model_92_05")

INFO:tensorflow:Assets written to: models/model_92_05/assets


INFO:tensorflow:Assets written to: models/model_92_05/assets


In [13]:
filtered_df = test_df[test_df['pred'] == 'r']
test_df[['label', 'pred']]

,label,pred
0,Test4751.png,2
1,Test2320.png,k
2,Test4989.png,N
3,Test6146.png,s
4,Test1629.png,s
...,...,...
7095,Test2339.png,P
7096,Test4990.png,N
7097,Test5456.png,9
7098,Test4748.png,0
